<a href="https://colab.research.google.com/github/JoelBenesh/m2pi.ca/blob/master/GHG_Emissions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
roundwood_bulking_factor = 0.6
chip_bulking_factor = 0.37
moisture_content = 0.55
dry_wood_weight = 120000
sawmill_waste_weight = 80000

average_distance = 53
truck_fuel_consumption = 56.7
diesel_carbon_content = 0.00269

work_days_per_year = 235

runtime_per_day = 10
grinder_fuel_consumption= 135

In [ ]:
slashpile_wood_composition = {
    "wood species" : ["slashpile %", "stem %", "stem calorific value (GJ/ODT)", "bark %", "bark calorific value (GJ/ODT)", "bulk density (kg/ m^3-solid)", "green density (kg/ m^3-solid)"],
    "cedar" : [0.1, 0.75, 20.2, .25, 21.1, 530],
    "pine" : [0.40, 0.75, 20.8, .25, 21.1, 420],
    "spruce" : [0.40, 0.75, 19.8, .25, 20.3, 450],
    "aspen" : [0.05, 0.9, 19.5, .1, 20, 420],
    "cottonwood" : [0.05, 0.9, 19.5, .1, 20, 410]
    }

for key in slashpile_wood_composition:
    if key != "wood species":
        wood_species = slashpile_wood_composition[key]
        bulk_density = wood_species[-1]  # Get the last element of the current list
        green_density = wood_species.append((bulk_density * roundwood_bulking_factor * 1.3) / (1 - moisture_content))

In [ ]:
for key in slashpile_wood_composition:
    print(slashpile_wood_composition[key])

['slashpile %', 'stem %', 'stem calorific value (GJ/ODT)', 'bark %', 'bark calorific value (GJ/ODT)', 'bulk density (kg/ m^3-solid)', 'green density (kg/ m^3-solid)']
[0.1, 0.75, 20.2, 0.25, 21.1, 530, 918.6666666666669]
[0.4, 0.75, 20.8, 0.25, 21.1, 420, 728.0000000000001]
[0.4, 0.75, 19.8, 0.25, 20.3, 450, 780.0000000000001]
[0.05, 0.9, 19.5, 0.1, 20, 420, 728.0000000000001]
[0.05, 0.9, 19.5, 0.1, 20, 410, 710.6666666666667]


In [ ]:
open_air_burning_emissions = {
    "gas" : ["emissions (T/DT)", "GWP"],
    "CO2" : [1.708, 1],
    "CO"  : [0.0663, 1],
    "CH4" : [0.005, 21],
    "N2O" : [0.00026, 310],
    "NOX" : [0.00103, 5]
    }

factory_burning_emissions = {
    "gas" : ["emissions (T/DT)", "GWP"],
    "CO2" : [1.715, 1],
    "CO"  : [0, 1],
    "CH4" : [0.0001, 21],
    "N2O" : [0.00007, 310],
    "NOX" : [0, 5]
    }

In [ ]:
def calculate_tCO2e(mass, emissions):
    """
    mass = the dry weight that is being burnt in tonnes
    emissions = dictionary consisting of a list with the gas emissions in tonnes per dry tonnes of emissions waste and global warming potential for each gas
    """
    tCO2e = 0
    for key in emissions:
        if key != "gas":
            tCO2e += mass * emissions[key][0] * emissions[key][1] # Sum the product of mass, emissions, and GWP for each gas
    return tCO2e

print(f"The dry mass emmissions for open air burning are given by: {calculate_tCO2e(dry_wood_weight, open_air_burning_emissions)}")
print(f"Comparatively, the dry mass emmissions for the factory controlled burning are given by: {calculate_tCO2e(dry_wood_weight, factory_burning_emissions)}")

The dry mass emmissions for open air burning are given by: 235806.0
Comparatively, the dry mass emmissions for the factory controlled burning are given by: 208656.0


In [ ]:
slashpile_weight_decomposition = {
    "wood species" : ["dry wood weight", "stem weight", "bark weight", "wet wood weight", "m3 solid round wet"]
}

for key in slashpile_wood_composition:
    if key != "wood species":
        weight_decomposition = []
        wood_species = slashpile_wood_composition[key]

        percent_in_slashpile = wood_species[0]
        weight_decomposition.append(percent_in_slashpile * dry_wood_weight)

        stem_percent = wood_species[1]
        weight_decomposition.append(percent_in_slashpile * dry_wood_weight * stem_percent)

        bark_percent = wood_species[3]
        weight_decomposition.append(percent_in_slashpile * dry_wood_weight * bark_percent)

        slashpile_weight_decomposition[key] = weight_decomposition

for key in slashpile_weight_decomposition:
    if key != "wood species":
        print(slashpile_weight_decomposition[key])

print(slashpile_weight_decomposition)

[12000.0, 9000.0, 3000.0]
[48000.0, 36000.0, 12000.0]
[48000.0, 36000.0, 12000.0]
[6000.0, 5400.0, 600.0]
[6000.0, 5400.0, 600.0]
{'wood species': ['dry wood weight', 'stem weight', 'bark weight', 'wet wood weight', 'm3 solid round wet'], 'cedar': [12000.0, 9000.0, 3000.0], 'pine': [48000.0, 36000.0, 12000.0], 'spruce': [48000.0, 36000.0, 12000.0], 'aspen': [6000.0, 5400.0, 600.0], 'cottonwood': [6000.0, 5400.0, 600.0]}


In [ ]:
for key in slashpile_weight_decomposition:
    if key != "wood species":
        weight_decomposition = slashpile_weight_decomposition[key]
        weight_decomposition.append(weight_decomposition[0]/(1 - moisture_content))

In [ ]:
wet_round_wood_volume = 0

for key in slashpile_weight_decomposition:
    if key != "wood species":
        weight_decomposition = slashpile_weight_decomposition[key]
        weight_decomposition.append(weight_decomposition[-1] * 1000/ slashpile_wood_composition[key][-1])

        wet_round_wood_volume += weight_decomposition[-1]

loose_wet_chips = wet_round_wood_volume * 60 / 37

print(f"Given that the dry slashpile mass of {dry_wood_weight}, the total wet chip volume would be : {loose_wet_chips}")

Given that the dry slashpile mass of 120000, the total wet chip volume would be : 566556.6549137649


In [ ]:
import math

truckloads = math.ceil(loose_wet_chips/170)

print(f"We would require {truckloads} to transport {loose_wet_chips} m^3 of loose chips with 55% moisture content or {dry_wood_weight} tonnes of dry wood. \n We note that this means there are roughly {math.ceil(truckloads/work_days_per_year)} truckloads per day")

def calculate_truckloads(mass, slashpile_composition):
    """
    This function calculates the number of truckloads required to transport the wet chips.

    mass = is the weight in tonnes
    slashpile_composition = a dictionary consisting of key:value pairs, where the key is a wood species and the values is a list of factors regarding the species.
    """


    return truckloads

We would require 3333 to transport 566556.6549137649 m^3 of loose chips with 55% moisture content or 120000 tonnes of dry wood. 
 We note that this means there are roughly 15 truckloads per day


In [ ]:
def calculate_transportation_emissions(distance, fuel_consumption_rate, fuel_carbon_content):
    """
    distance = average distance travelled in km.
    fuel_consumption_rate = truck's recorded annual fuel consumption in L/100km.
    fuel_carbon_content = the average fuel carbon content, ~ 2.69 kg CO2/L or 0.00268 t CO2/L.
    """
    fuel_consumption = (distance/ 100) * fuel_consumption_rate # in litres. NB : We divide this by 100 so that the distance is in 100km allowing cancellations
    carbon_emissions_tCO2e = fuel_consumption * fuel_carbon_content # in kg CO2
    return carbon_emissions_tCO2e

print(f"These transport emissions would cause roughly {2 * truckloads * calculate_transportation_emissions(average_distance, truck_fuel_consumption, diesel_carbon_content)} tonnes of CO2 equivalent for annum.")

These transport emissions would cause roughly 538.86070854 tonnes of CO2 equivalent for annum.


In [ ]:
def calculate_grinder_emissions(time, fuel_consumption_rate, fuel_carbon_content):
    """
    time = time in hours.
    fuel_consumption_rate = grinder's recorded annual fuel consumption in L/hr
    fuel_carbon_content = the average fuel carbon content, ~ 2.69 kg CO2/L or 0.00268 t CO2/L.
    """
    fuel_consumption = time * fuel_consumption_rate # in litres
    carbon_emissions_tCO2e = fuel_consumption * fuel_carbon_content # in kg CO2
    return carbon_emissions_tCO2e

print(f"These grinder emissions would cause roughly {work_days_per_year * calculate_grinder_emissions(runtime_per_day, grinder_fuel_consumption, diesel_carbon_content)} tonnes of CO2 equivalent for annum.")

These grinder emissions would cause roughly 853.4025 tonnes of CO2 equivalent for annum.


In [ ]:
def calculate_factory_efficiency(energy_output, biomass, sawmill_waste):
    """
    energy_output = the factory's energy output in Mega Watt Hours
    biomass = the dry weight in tonnes of biomass
    sawmill_waste = the mass of sawmill waste product in tonnes being combined with biomass for combustion
    """
    total_mass = biomass + sawmill_waste
    wood_species_potential_energy = {"wood species" : ["dry weight", "stem weight", "bark weight", "stem energy", "bark energy"]}
    for key in slashpile_wood_composition:
        if key != "wood species":

    stem_weight =

IndentationError: ignored